In [1]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

In [2]:
import os
print(os.environ['GOOGLE_API_KEY'][:10]+'*'*10)

AIzaSyC06D**********


In [3]:
from langchain_teddynote import logging

logging.langsmith("Synthetic Data Generation")

LangSmith 추적을 시작합니다.
[프로젝트명]
Synthetic Data Generation


In [4]:
from langchain_community.document_loaders import PDFPlumberLoader

# 문서 로더 생성
loader = PDFPlumberLoader("data/Domain-specialized-LLM-Financial-fine-tuning-and-utilization-method-using-Mistral-7B.pdf")

# 문서 로딩
docs = loader.load()

# 목차, 끝 페이지 제외
docs = docs[3:-1]

# 문서의 페이지수
len(docs)

25

In [5]:
docs[0].metadata

{'source': 'data/Domain-specialized-LLM-Financial-fine-tuning-and-utilization-method-using-Mistral-7B.pdf',
 'file_path': 'data/Domain-specialized-LLM-Financial-fine-tuning-and-utilization-method-using-Mistral-7B.pdf',
 'page': 3,
 'total_pages': 29,
 'Author': 'MS',
 'CreationDate': "D:20240328014510+09'00'",
 'Creator': 'PScript5.dll Version 5.2.2',
 'ModDate': "D:20240328014510+09'00'",
 'Producer': 'Acrobat Distiller 9.3.2 (Windows)',
 'Title': '<C1F6B4C9C1A4BAB8203330B1C731C8A35FC5EBBABB2E687770>',
 'rgid': 'PB:379341632_AS:11431281232126100@1711621456089'}

In [6]:
# metadata 설정(filename 이 존재해야 함)
for doc in docs:
    doc.metadata["filename"] = doc.metadata["source"]

In [27]:
from ragas.testset import TestsetGenerator
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper

from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_ollama import OllamaEmbeddings

# 데이터셋 생성기
# llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash")
llm = ChatOpenAI(model="gpt-4o-mini")

# 문서 임베딩
# embeddings = OllamaEmbeddings(model="bge-m3")
embeddings = OllamaEmbeddings(model="bge-m3")


In [28]:
# LangChain의 gemini 모델을 LangchainLLMWrapper로 감싸 Ragas와 호환되게 만듭니다.
generator_llm = LangchainLLMWrapper(llm)

# ragas_embeddings 생성
ragas_embeddings = LangchainEmbeddingsWrapper(embeddings)


In [ ]:
generator = TestsetGenerator(llm = generator_llm, embedding_model=ragas_embeddings)
dataset = generator.generate_with_langchain_docs(docs, testset_size=10, with_debugging_logs=True,)

In [ ]:
# 생성된 테스트셋을 pandas DataFrame으로 변환
test_df = dataset.to_pandas()
test_df

## deeper

In [29]:
from ragas.testset.graph import KnowledgeGraph

kg = KnowledgeGraph()

In [30]:
from ragas.testset.graph import Node, NodeType

for doc in docs:
    kg.nodes.append(
        Node(
            type=NodeType.DOCUMENT,
            properties={"page_content": doc.page_content, "document_metadata": doc.metadata}
        )
    )

In [31]:
kg

KnowledgeGraph(nodes: 25, relationships: 0)

In [32]:
from ragas.testset.transforms import default_transforms, apply_transforms


# define your LLM and Embedding Model
# here we are using the same LLM and Embedding Model that we used to generate the testset
transformer_llm = generator_llm
embedding_model = LangchainEmbeddingsWrapper(OllamaEmbeddings(model="bge-m3"))

trans = default_transforms(documents=docs, llm=transformer_llm, embedding_model=embedding_model)
apply_transforms(kg, trans)

Applying HeadlinesExtractor:   0%|          | 0/25 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/25 [00:00<?, ?it/s]

Applying SummaryExtractor:   0%|          | 0/50 [00:00<?, ?it/s]

Property 'summary' already exists in node '9a722a'. Skipping!
Property 'summary' already exists in node '8db949'. Skipping!
Property 'summary' already exists in node 'e95bb2'. Skipping!
Property 'summary' already exists in node 'c5f2cc'. Skipping!
Property 'summary' already exists in node '2c8b3c'. Skipping!
Property 'summary' already exists in node 'f38220'. Skipping!
Property 'summary' already exists in node '0f10fa'. Skipping!
Property 'summary' already exists in node '64a1f3'. Skipping!
Property 'summary' already exists in node '7937c4'. Skipping!
Property 'summary' already exists in node '9be4ed'. Skipping!
Property 'summary' already exists in node '5f5841'. Skipping!
Property 'summary' already exists in node '68b3a2'. Skipping!
Property 'summary' already exists in node 'c49902'. Skipping!
Property 'summary' already exists in node '2d5152'. Skipping!
Property 'summary' already exists in node '7ca2b8'. Skipping!
Property 'summary' already exists in node 'b46b0b'. Skipping!
Property

Applying CustomNodeFilter: 0it [00:00, ?it/s]

Applying [EmbeddingExtractor, ThemesExtractor, NERExtractor]:   0%|          | 0/50 [00:00<?, ?it/s]

Property 'summary_embedding' already exists in node 'c5f2cc'. Skipping!
Property 'summary_embedding' already exists in node '9a722a'. Skipping!
Property 'summary_embedding' already exists in node '8db949'. Skipping!
Property 'summary_embedding' already exists in node 'e95bb2'. Skipping!
Property 'summary_embedding' already exists in node '2c8b3c'. Skipping!
Property 'summary_embedding' already exists in node '703be1'. Skipping!
Property 'summary_embedding' already exists in node 'f38220'. Skipping!
Property 'summary_embedding' already exists in node 'd9180e'. Skipping!
Property 'summary_embedding' already exists in node '7937c4'. Skipping!
Property 'summary_embedding' already exists in node '9be4ed'. Skipping!
Property 'summary_embedding' already exists in node '0f10fa'. Skipping!
Property 'summary_embedding' already exists in node '5f5841'. Skipping!
Property 'summary_embedding' already exists in node '64a1f3'. Skipping!
Property 'summary_embedding' already exists in node 'c49902'. Sk

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

In [33]:
kg

KnowledgeGraph(nodes: 50, relationships: 493)

In [34]:
kg.save("knowledge_graph.json")

In [35]:
loaded_kg = KnowledgeGraph.load("knowledge_graph.json")
loaded_kg

KnowledgeGraph(nodes: 50, relationships: 493)

In [36]:
from ragas.testset.synthesizers import default_query_distribution

query_distribution = default_query_distribution(generator_llm)

In [44]:
from ragas.testset import TestsetGenerator
embedding_model = LangchainEmbeddingsWrapper(OpenAIEmbeddings(model="text-embedding-3-small"))

generator = TestsetGenerator(llm=generator_llm, embedding_model=embedding_model, knowledge_graph=loaded_kg)

In [46]:

testset = generator.generate(testset_size=10)

Generating personas:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/2 [00:00<?, ?it/s]

Generating Samples: 0it [00:00, ?it/s]

In [47]:
test_df = testset.to_pandas()